# KoGPT-2를 활용한 Few-shot & Zero-shot 실습


> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

Transformers를 설치하고 Model을 load하겠습니다.

In [ ]:
!pip install transformers

     |████████████████████████████████| 1.9MB 15.5MB/s 
     |████████████████████████████████| 3.2MB 51.9MB/s 
     |████████████████████████████████| 890kB 47.9MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=34b460eb6261f65ebbc7e26b3456c1f7208b90b2314423854dd048c92d611bb2
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Detected operating system as Ubuntu/bionic.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... done.
Running apt-get update... done.

The repository is setup! You can now install packages.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 46 not upgraded.
Need to get 6,540 kB of archives.
After this operation, 15.3 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu bionic/main amd64 git-lfs amd64 2.13.2 [6,540 kB]
Fetched 6,540 kB in 1s (7,831 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 160979 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.13.2_a

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
config.pad_token_id = tokenizer.token_to_id('<pad>')
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50000, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): Laye

In [ ]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [ ]:
tokenizer.add_special_tokens(["<s>", "</s>"])

0

In [ ]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

GENERATED SEQUENCE : 이순신은 백옥담(이종원 분)의 시선으로 백성을 살피는 것을 알게 되었고, 천하에 대한 소신을 가지게 되었다. 또한 성읍인 성전은 백옥담의 동생인 윤(尹)이 성전과 함께 있었는데 윤을 그의 아버지라고 불렀다. 또한 성전이 완공될 때까지 성전에서 계속 놀 수 있었다. 성전 안에서는 두 권의 도서가 있고, 이 문서에는 윤과 성국의 성전의 위치가 찍혀져 있다.
GENERATED SEQUENCE : 이순신은 이미 태연에게 마음을 돌린 상태. 이어 "자칫 북한이 미사일 도발에 나설 수도 있다는 경고가 되고 있는 만큼 한미가 이에 대해 적절히 대응해주길 바란다"고 당부했다. 또한 "오늘 김관진 제1위원장이 오랫만에 공개토론을 한 뒤 북한 도발을 중지하고 대화를 시작해야 한다고 강조한 것을 감안할 때 이번 주 미측과 한반도에서의 대화 분위기가 계속될 것으로 확신한다"고 말했다. 또한 김 제1위원장의 이번 방문에서 미국측에 한반도 신뢰프로세스 차원의
GENERATED SEQUENCE : 이순신은 고려 태조 이성계를 도와 이성계 형진을 돕고 북경을 약탈케 했다. 따라서, 그의 아들들 중에는 아누비스의 딸에게 왕위를 넘겨준 후(그녀에게 왕위를 주지 못한 후)나, 아버지에게 버림받은 후(그녀)가 왕좌에 오를 수 있었던 것을 보아야만 했다. 1017년경, 아버지 레오나르가 사보이 왕국을 지배하게 되면서, 그의 후계자들은 모국(바빌론 왕국과 알


In [ ]:
def get_gpt_output(input_sent):
    input_ids = torch.tensor(tokenizer.encode(input_sent, add_special_tokens=True).ids).unsqueeze(0).to('cuda')
    sample_outputs = model.generate(
        input_ids,
        do_sample=True, 
        max_length=512, 
        top_k=50, 
        top_p=0.95, 
        num_return_sequences=1,
        eos_token_id=tokenizer.token_to_id("</s>"),
        no_repeat_ngram_size=2,
        early_stopping=True
    )
    generated_sequence = sample_outputs[0].tolist()
    return tokenizer.decode(generated_sequence, skip_special_tokens=True)

In [ ]:
get_gpt_output("<s>이순신은")

'이순신은 극중 인현왕후(박지영)에게 혼담 초반 당황한 마음을 주체하지 못하고, “나는 그 순간부터 내가 미쳤구나, 그런 느낌을 가질 것이다”라고 말하며 그녀를 당황케 했다.'

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : ")

'철수 : 영희야 안녕! 영희 : ! - ^^* 아!!'

In [ ]:
get_gpt_output("<s>철수 : 영희야 안녕!</s><s>영희 : 어! 철수야! 오랜만이다!</s><s>철수 : 그러게~ 잘 지냈어?</s><s>영희 : ")

'철수 : 영희야 안녕! 영희 : 어! 철수야! 오랜만이다! 철수 : 그러게~ 잘 지냈어? 영희 : 훗, 잘 지내지? - 난 오늘도 오늘도 학교 안 가네 - 수진 : 무슨?!- 영 희진아 뭐 해? : 네, 괜찮아요.'

In [ ]:
get_gpt_output("<s>본문 : 아.. 기분 진짜 짜증나네ㅡㅡ</s><s>감정 : 분노</s><s>본문 : 와!! 진짜 너무 좋아!!</s><s>감정 : ")

'본문 : 아.. 기분 진짜 짜증나네 감정 : 분노 본문 : 와!! 진짜 너무 좋아!! 감정 : ♥.'

In [ ]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 인공지능의 미래에 대해 어떻게 생각하십니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 인공지능의 미래에 대해 어떻게 생각하십니까? 답 : rain_NVS가 현재 개발 중인 urgical music control quant cruisine에 의해 연구된다면 인공지능이 과연 인공지능에게 얼마나 중요한 의미를 가지는 지를 보여줌니다. money에 대한 자신의 생각을 말하기 위해서입니다.]'

In [ ]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 자연어처리에서 언어모델이란 무엇입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 자연어처리에서 언어모델이란 무엇입니까? 답 : operator(비언어 모델)은 언어를 포함한 비언어 모델의 모델(가성언어)에서 발생하는 것을 의미합니다. (plaster)는 단어, 단어 및 그 이외의 모든 언어가 사용되는 언어 모델 (나머지 언어 모델을 포함)의 하나로 언어학적 문제 발생 시 그 현상을 파악하는 데 활용됩니다.('

In [ ]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 대한민국에서 최고의 인공지능 기술을 보유한 기업은 어디입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 대한민국에서 최고의 인공지능 기술을 보유한 기업은 어디입니까? 답 : 렉스 테크놀러지가 개발한 인공신경분석 기술을 가진 기업입니다..'

In [ ]:
get_gpt_output("<s>질문 : 코로나 바이러스에 걸리면 어떻게 되나요?</s>\
<s>답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다.</s>\
<s>질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까?</s>\
<s>답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다.</s>\
<s>질문 : 딸기 식물의 수명주기는 무엇입니까?</s>\
<s>답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다.</s>\
<s>질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까?</s>\
<s>답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다.</s>\
<s>질문 : 뇌의 어떤 부분이 말을 제어합니까?</s>\
<s>답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다.</s>\
<s>질문 : 이순신 장군이 전사한 전투는 무슨 전투입니까?</s>\
<s>답 : ")

'질문 : 코로나 바이러스에 걸리면 어떻게 되나요? 답 : COVID-19 환자는 일반적으로 감염 후 평균 5 ~ 6 일 (평균 잠복기 5 ~ 6 일, 범위 1 ~ 14 일)에 경미한 호흡기 증상 및 발열을 포함한 징후와 증상을 나타냅니다. COVID-19 바이러스에 감염된 대부분의 사람들은 경미한 질병을 앓고 회복됩니다. 질문 : 코로나 바이러스 질병의 첫 증상은 무엇입니까? 답 : 이 바이러스는 경미한 질병에서 폐렴에 이르기까지 다양한 증상을 유발할 수 있습니다. 질병의 증상은 발열, 기침, 인후통 및 두통입니다. 심한 경우 호흡 곤란과 사망이 발생할 수 있습니다. 질문 : 딸기 식물의 수명주기는 무엇입니까? 답 : 딸기의 생애는 새로운 식물의 설립으로 시작하여 2 ~ 3 년 후 절정에 이르렀다가 절정에 이어 2 ~ 3 년에 노화와 죽음을 향해 진행됩니다. 이상적인 조건에서 딸기 식물은 5-6 년까지 살 수 있습니다. 질문 : 파이썬 메서드의 self 매개 변수의 목적은 무엇입니까? 답 : self 매개 변수는 클래스의 현재 인스턴스에 대한 참조이며 클래스에 속한 변수에 액세스하는 데 사용됩니다. 질문 : 뇌의 어떤 부분이 말을 제어합니까? 답 : 언어 우세 반구의 왼쪽 전두엽 (브로카 영역)에있는 뇌의 분리 된 부분에 대한 손상은 자발적 언어 및 운동 언어 제어 사용에 상당한 영향을 미치는 것으로 나타났습니다. 질문 : 이순신 장군이 전사한 전투는 무슨 전투입니까? 답 : ×대장 (사부 전선) 이 전투에서 두명의 전사자를 포함하여 6명이 전사했습니다. ...'

In [ ]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 이순신은 조선 중기의 무신이다.</s>\
<s>Englisth: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 이순신은 조선 중기의 무신이다. Englisth: artery in an evasivation of the magical science and materials much of England of society. eurology of Technology. ( ) Ilvironment all after) Engleish and Industry induction. -_ Intario. _( ) Enkult Engal

In [ ]:
get_gpt_output("<s>한국어: 그 도로는 강과 평행으로 뻗어 있다.</s>\
<s>English: The road runs parallel to the river.</s>\
<s>한국어: 그 평행선들은 분기하는 것처럼 보인다.</s>\
<s>English: The parallel lines appear to diverge.</s>\
<s>한국어: 그 도로와 운하는 서로 평행하다.</s>\
<s>English: The road and the canal are parallel to each other.</s>\
<s>한국어: 평행한 은하계라는 개념은 이해하기가 힘들다.</s>\
<s>English: The idea of a parallel universe is hard to grasp.</s>\
<s>한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다.</s>\
<s>English: This tradition has no parallel in our culture.</s>\
<s>한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다.</s>\
<s>English: This is an achievement without parallel in modern times.</s>\
<s>한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다.</s>\
<s>English: It is possible to draw a parallel between their experience and ours.</s>\
<s>한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다.</s>\
<s>English: The new degree and the existing certificate courses would run in parallel.</s>\
<s>한국어: 100명은 서 있을 수 있을 것 같은 섬 하나가 넓은 하늘 위에 두둥실 떠 있다.</s>\
<s>Englisth: ")

'한국어: 그 도로는 강과 평행으로 뻗어 있다. English: The road runs parallel to the river. 한국어: 그 평행선들은 분기하는 것처럼 보인다. English: The parallel lines appear to diverge. 한국어: 그 도로와 운하는 서로 평행하다. English: The road and the canal are parallel to each other. 한국어: 평행한 은하계라는 개념은 이해하기가 힘들다. English: The idea of a parallel universe is hard to grasp. 한국어: 이러한 전통은 우리 문화에서는 그에 상응하는 것이 없다. English: This tradition has no parallel in our culture. 한국어: 이것은 현대에 들어서는 그 유례를 찾기 힘든 업적이다. English: This is an achievement without parallel in modern times. 한국어: 그들의 경험과 우리 경험 사이에서 유사점을 찾는 것이 가능하다. English: It is possible to draw a parallel between their experience and ours. 한국어: 그 새 학위 과정과 기존의 수료 과정이 동시에 운영될 수도 있을 것이다. English: The new degree and the existing certificate courses would run in parallel. 한국어: 100명은 서 있을 수 있을 것 같은 섬 하나가 넓은 하늘 위에 두둥실 떠 있다. Englisth: appsue remixments him.exporing the area of refers and spectuated plans on see the decided first ever crisissession.Eximations interested adiente to mayen feet.A.H.C.S.B.D.U.M.